In [2]:
import csv
import os
import numpy as np
import pandas as pd
import sklearn.metrics as mm
import sys
# sir_inference imports
from sir_model import FastProximityModel, patient_zeros_states
from ranking import csr_to_list
import os.path
from os import path
N=500000
## new try with 100 spreaders
N_patient_zero = 200;
lamb_load = 0.03;
lamb = 0.05
mu = 0.02;
scale=1.0; # Easy Case
T=100;
seed=32
location="networks"
# SIR parameters
t1 = 10;
#seed=int();

In [3]:
## import sys
from time import time
import numpy as np
import pandas as pd
import os.path
from os import path
from scenario import Scenario
from sir_model import EpidemicModel, patient_zeros_states, symptomatic_individuals
from ranking import RANKINGS
log_fname="interactions_proximity_N%dK_s%.1f_T%d_lamb%.2f_s%d.csv"%(N/1000,scale,T,lamb_load,seed)
csv_file=location+"/"+log_fname
print("seed = %d"%seed, flush=True)
np.random.seed(seed);
if path.exists(csv_file): print("Loading model from %s"%csv_file)
else:
    print("Could not find the file! Was looking for \n "+csv_file+"\n Bye-Bye")
    sys.exit()
print("Load Proximity model", flush=True)
tic = time()
model = EpidemicModel(initial_states=np.zeros(N), x_pos=np.zeros(N), y_pos=np.zeros(N))
model.load_transmissions(csv_file, new_lambda = lamb)
model.recover_probas = mu*np.ones(N)
print(f"Loading took {time()-tic:.1f}s",flush=True)
model.initial_states = patient_zeros_states(N, N_patient_zero)
model.time_evolution(model.recover_probas, model.transmissions, print_every=50)
t_max = len(model.transmissions)
print("Save plain dynamics", flush=True)
db = pd.DataFrame()
db["S"] = np.sum(model.states==0,axis=1)
db["I"] = np.sum(model.states==1,axis=1)
db["R"] = np.sum(model.states==2,axis=1)
db.to_csv("csv/Proximity_N%dK_T%d_s1_pz%d_mu%.2f_l%.2f_seed%d.csv"%(N/1000,T,N_patient_zero,mu,lamb,seed),
          index=False, sep="\t")
del db
model.initial_states = model.states[t1]
model.states = model.states[t1:]
model.transmissions = model.transmissions[t1:]

seed = 32
Loading model from networks/interactions_proximity_N500K_s1.0_T100_lamb0.03_s32.csv
Load Proximity model
Loading transmissions from networks/interactions_proximity_N500K_s1.0_T100_lamb0.03_s32.csv
Loading took 310.4s
Running SIR simulation
t = 0 / 100
t = 50 / 100
Save plain dynamics


In [4]:
# trac parameters
trac_tau = 5;
# MF parameters
MF_tau = 5;
MF_delta = 15;
# observation parameters
n_ranking = 1500
p_untracked=0
#seed = int(sys.argv[2]);
#seeds for running [32,123,456]
#seed=int(sys.argv[1]);
################################################

intervention_options=dict(quarantine_time=T-t1)
observation_options=dict(n_random=0,n_infected=0,n_ranking=n_ranking, p_symptomatic=0.5, tau=5, p_untracked=p_untracked)

In [5]:
##### MF SCENARIO #######
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
import ranking
imp.reload(ranking)
from ranking import RANKINGS
scenario_MF = Scenario(
    model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["backtrack"], 
                         algo="MF", init="all_S", tau=MF_tau, delta=MF_delta), 
    observation_options=observation_options,
    intervention_options=intervention_options,
)
scenario_MF.run(t_max-t1, print_every = 1)
print("Save MF strategy", flush=True)
scenario_MF.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_MF_t%d_d%d.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,MF_tau,MF_delta),
                  index=False, sep="\t")
del scenario_MF
# 1h01min per round
print("End seed", flush=True)

Using ranking_backtrack to rank
Using seed=33
Selecting 0/500000 untracked individuals
t = 0 / 90
state: S:497746, I:2091, R:163 I+R:2254
obser: S:1494, I:4, R:2, I+R:6
free birds: 2087
obs_symp:0(0), obs_rank:4(4)
t = 1 / 90
state: S:497295, I:2506, R:199 I+R:2705
obser: S:2986, I:10, R:4, I+R:14
free birds: 2496
obs_symp:0(0), obs_rank:6(10)
t = 2 / 90
state: S:496742, I:3005, R:253 I+R:3258
obser: S:4427, I:62, R:11, I+R:73
free birds: 2943
obs_symp:0(0), obs_rank:52(62)
t = 3 / 90
state: S:496142, I:3537, R:321 I+R:3858
obser: S:5691, I:280, R:29, I+R:309
free birds: 3260
obs_symp:0(0), obs_rank:218(280)
t = 4 / 90
state: S:495496, I:4107, R:397 I+R:4504
obser: S:7088, I:378, R:34, I+R:412
free birds: 3737
obs_symp:0(0), obs_rank:98(378)
t = 5 / 90
state: S:494763, I:4762, R:475 I+R:5237
obser: S:8580, I:385, R:35, I+R:420
free birds: 4390
obs_symp:0(0), obs_rank:7(385)
t = 6 / 90
state: S:493885, I:5542, R:573 I+R:6115
obser: S:10065, I:581, R:35, I+R:616
free birds: 4982
obs_symp

In [6]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
##### RANDOM SCENARIO #######
scenario_rnd = Scenario(
    model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["random"]),
    observation_options=observation_options,
    intervention_options=intervention_options
)
scenario_rnd.run(t_max-t1,  print_every = 1)
print("Save random strategy", flush=True)
scenario_rnd.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_rnd.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking),
          index=False, sep="\t")
del scenario_rnd

Using ranking_random to rank
Using seed=33
Selecting 0/500000 untracked individuals
t = 0 / 90
state: S:497746, I:2091, R:163 I+R:2254
obser: S:1496, I:4, R:0, I+R:4
free birds: 2087
obs_symp:0(0), obs_rank:4(4)
t = 1 / 90
state: S:497293, I:2508, R:199 I+R:2707
obser: S:2984, I:14, R:2, I+R:16
free birds: 2494
obs_symp:0(0), obs_rank:10(14)
t = 2 / 90
state: S:496739, I:3008, R:253 I+R:3261
obser: S:4474, I:24, R:2, I+R:26
free birds: 2985
obs_symp:0(0), obs_rank:10(24)
t = 3 / 90
state: S:496131, I:3548, R:321 I+R:3869
obser: S:5959, I:39, R:2, I+R:41
free birds: 3511
obs_symp:0(0), obs_rank:15(39)
t = 4 / 90
state: S:495438, I:4165, R:397 I+R:4562
obser: S:7452, I:46, R:2, I+R:48
free birds: 4122
obs_symp:0(0), obs_rank:7(46)
t = 5 / 90
state: S:494635, I:4891, R:474 I+R:5365
obser: S:8926, I:70, R:4, I+R:74
free birds: 4824
obs_symp:0(0), obs_rank:24(70)
t = 6 / 90
state: S:493653, I:5777, R:570 I+R:6347
obser: S:10406, I:289, R:6, I+R:295
free birds: 5493
obs_symp:206(206), obs_ra

In [ ]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
import sib
import bp_ranking
imp.reload(bp_ranking)
from bp_ranking import bp_ranker_class

mu_r = np.log(1 + mu)
name_csv = "csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_bp%d_no_tau.csv"%(N, T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau)
pseed=10/N
psus = 0.55
pautoinf = 1e-10
bp_ranker = bp_ranker_class(params = sib.Params(
                                 prob_r = sib.Exponential(mu=mu_r),
                                 pseed = pseed,
                                 psus = psus,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                            tau=10
                           )
bp_ranker.init(N, T)
bp_ranker.__name__ = "bp"
scenario_bp = Scenario( model, seed=seed+1, 
    ranking_options=dict(ranking=bp_ranker.step_scenario),
    observation_options=observation_options,
    intervention_options=intervention_options,
                         save_csv = name_csv
)
scenario_bp.run(t_max-t1, print_every = 1)
print("Save bp strategy", flush=True)
scenario_bp.counts.to_csv(name_csv,
          index=False, sep="\t")


Using step_scenario to rank
Using seed=33
Selecting 0/500000 untracked individuals

SCENARIO BP
t = 0 / 90
state: S:497746, I:2091, R:163 I+R:2254
obser: S:1494, I:4, R:2, I+R:6
free birds: 2087
obs_symp:0(0), obs_rank:4(4)

SCENARIO BP
Observations: 1 obs 1500 1500 0
sib.iterate(damp=0):  2/20 1.048e-05/0.001    
sib.iterate(damp=0.5):  0/20 7.575e-07/0.001    
winBP: (S,I,R): (499972.7, 24.5, 2.8), seeds: 23.2, ll: -298970.6
t = 1 / 90
state: S:497295, I:2506, R:199 I+R:2705
obser: S:2982, I:16, R:2, I+R:18
free birds: 2490
obs_symp:0(0), obs_rank:12(16)

SCENARIO BP
Observations: 2 obs 1500 1500 0
sib.iterate(damp=0):  5/20 3.114e-04/0.001    
sib.iterate(damp=0.5):  0/20 1.124e-05/0.001    
winBP: (S,I,R): (499945.1, 50.9, 4.1), seeds: 34.0, ll: -299091.3
t = 2 / 90
state: S:496744, I:3003, R:253 I+R:3256
obser: S:4187, I:289, R:24, I+R:313
free birds: 2715
obs_symp:0(0), obs_rank:273(289)

SCENARIO BP
Observations: 3 obs 1500 1500 0
sib.iterate(damp=0):  19/20 1.280e-01/0.001    


In [ ]:
##### TRACING SCENARIO #######
scenario_trac = Scenario( model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["tracing"], tau=trac_tau),
    observation_options=observation_options,
    intervention_options=intervention_options
)
scenario_trac.run(t_max-t1, print_every = 1)
print("Save tracing strategy", flush=True)
scenario_trac.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_trac_t%d.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau),
                  index=False, sep="\t")
del scenario_trac

In [7]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
import sib
import bp_ranking
imp.reload(bp_ranking)
from bp_ranking import bp_ranker_class

mu_r = np.log(1 + mu)
name_csv = "csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_bp%d.csv"%(N, T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau)
pseed=10/N
psus = 0.60
pautoinf = 1e-10
bp_ranker = bp_ranker_class(params = sib.Params(
                                 prob_r = sib.Exponential(mu=mu_r),
                                 pseed = pseed,
                                 psus = psus,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                            tau=7
                           )
bp_ranker.init(N, T)
bp_ranker.__name__ = "bp"
scenario_bp = Scenario( model, seed=seed, 
    ranking_options=dict(ranking=bp_ranker.step_scenario),
    observation_options=observation_options,
    intervention_options=intervention_options,
                         save_csv = name_csv
)
scenario_bp.run(t_max-t1, print_every = 1)
print("Save bp strategy", flush=True)
scenario_bp.counts.to_csv(name_csv,
          index=False, sep="\t")


Using step_scenario to rank
Using seed=2
Selecting 0/50000 untracked individuals
SCENARIO BP
t = 0 / 90
state: S:49928, I:68, R:4 I+R:72
obser: S:100, I:0, R:0, I+R:0
free birds: 68
obs_symp:0(0), obs_rank:0(0)
SCENARIO BP
Observations: 1 obs 100 100 0
sib.iterate(damp=0):  2/20 5.992e-05/0.001    
sib.iterate(damp=0.5):  0/20 1.556e-06/0.001    
winBP: (S,I,R): (49982.8, 16.9, 0.3), seeds: 16.0, ll: -25525.3
t = 1 / 90
state: S:49919, I:77, R:4 I+R:81
obser: S:199, I:1, R:0, I+R:1
free birds: 76
obs_symp:0(0), obs_rank:1(1)
SCENARIO BP
Observations: 2 obs 100 100 0
sib.iterate(damp=0):  4/20 1.822e-04/0.001    
sib.iterate(damp=0.5):  0/20 1.348e-05/0.001    
winBP: (S,I,R): (49980.9, 18.6, 0.5), seeds: 16.3, ll: -25533.9
t = 2 / 90
state: S:49912, I:83, R:5 I+R:88
obser: S:289, I:11, R:0, I+R:11
free birds: 72
obs_symp:0(0), obs_rank:10(11)
SCENARIO BP
Observations: 3 obs 100 100 0
sib.iterate(damp=0):  11/20 8.910e-04/0.001    
sib.iterate(damp=0.5):  0/20 5.311e-04/0.001    
winBP:

KeyboardInterrupt: 

In [ ]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
#imp.reload(sib)
import sib
import bp_ranking
imp.reload(bp_ranking)
from bp_ranking import bp_ranker_class

mu_r = np.log(1 + mu)
name_csv = "csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_bp%d_tau_obs.csv"%(N, T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau)
pseed=10/N
psus = 0.60
pautoinf = 1e-10
bp_ranker = bp_ranker_class(params = sib.Params(
                                 prob_r = sib.Exponential(mu=mu_r),
                                 pseed = pseed,
                                 psus = psus,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                           )
bp_ranker.init(N, T)
bp_ranker.set_tau_obs(5)
bp_ranker.__name__ = "bp"
scenario_bp = Scenario( model, seed=seed, 
    ranking_options=dict(ranking=bp_ranker.step_scenario),
    observation_options=observation_options,
    intervention_options=intervention_options,
                         save_csv = name_csv
)
scenario_bp.run(t_max-t1, print_every = 1)
print("Save bp strategy", flush=True)
scenario_bp.counts.to_csv(name_csv,
          index=False, sep="\t")


In [ ]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
import sib
import bp_ranking
imp.reload(bp_ranking)
from bp_ranking import bp_ranker_class

mu_r = np.log(1 + mu)
name_csv = "csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_bp%d_no_tau.csv"%(N, T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau)
pseed=10/N
psus = 0.60
pautoinf = 1e-10
bp_ranker = bp_ranker_class(params = sib.Params(
                                 prob_r = sib.Exponential(mu=mu_r),
                                 pseed = pseed,
                                 psus = psus,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                           )
bp_ranker.init(N, T)
bp_ranker.__name__ = "bp"
scenario_bp = Scenario( model, seed=seed, 
    ranking_options=dict(ranking=bp_ranker.step_scenario),
    observation_options=observation_options,
    intervention_options=intervention_options,
                         save_csv = name_csv
)
scenario_bp.run(t_max-t1, print_every = 1)
print("Save bp strategy", flush=True)
scenario_bp.counts.to_csv(name_csv,
          index=False, sep="\t")
